# <font color=purple><center>Quem classifica a classe classificadora?
    
### <font color=violet><center>Maria Clara Macêdo Lelis
    
<font color=green> Neste notebook é implementada uma rede MLP de classificação em python puro, ela é aplicada aos dados do dataset `geiyser` da biblioteca `seaborn` é feita uma classificação binária dos geisers em longo(1) e curto(0).
    
    
<font color=green>Primeiramente importamos as bibliotecas a serem utilizadas. Após isso carregamos o dataset `geyser`, realizamos uma normaluzação dos dados e fazemos um split de treino e teste.  


In [9]:
import random
import math
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

geyser=sns.load_dataset("geyser").dropna()

labels=geyser["kind"].map({"short": 0, "long": 1}).values
features=geyser[["duration", "waiting"]].values

scaler=StandardScaler()
features=scaler.fit_transform(features)

X_train, X_test, y_train, y_test=train_test_split(features, labels, test_size=0.1, random_state=280106)

<font color=green>Então definimos a classe `Valor`, que representa um número com operações personalizadas e suporte a gradiente.Ela armazena um valor escalar (data) e seu gradiente (grad).
Sobrecarga os operadores `+` e `*` para permitir contas entre objetos Valor e floats/ints.
Inclui a função `sig()` que aplica a ativação sigmoide, útil para redes neurais.
Serve como base para construir uma rede neural artesanal com controle sobre os dados e gradientes.

In [2]:
class Valor:
    def __init__(self, data):
        self.data=data
        self.grad=0.0

    def __add__(self, other):
        other=other if isinstance(other, Valor) else Valor(other)
        out=Valor(self.data + other.data)
        return out

    def __mul__(self, other):
        other=other if isinstance(other, Valor) else Valor(other)
        out=Valor(self.data * other.data)
        return out

    def __radd__(self, other): return self + other
    def __rmul__(self, other): return self * other

    def sig(self):
        x = self.data
        return Valor(1/(1 + math.exp(-x)))

    def __repr__(self):
        return f"Valor({self.data:.4f})"

<font color=green>  O próximo passo é criar a classe `Neuronio` que representa os neurônios artificiais que serão usados na rede. Ele recebe entradas, multiplica cada uma por um peso (escolhido aleatoriamente), soma tudo com um viés (bias) e aplica uma função de ativação, nesse caso a função sigmoide.

In [3]:
class Neuronio:
    def __init__(self, entradas):
        self.pesos=[Valor(random.uniform(-1, 1)) for _ in range(entradas)]
        self.bias=Valor(0.0)

    def __call__(self, x):
        soma=sum((w * xi for w, xi in zip(self.pesos, x)), self.bias)
        return soma.sig() 

<font color=green> Agora criamos a classe `Camada` que representa as camadas da rede. Ao ser criada, a camada recebe o número de entradas e o número de saídas desejado, e então cria um neurônio para cada saída. Quando a camada é chamada com uma lista de entradas, ela envia essas entradas para cada neurônio e retorna uma lista com as saídas de todos eles.

In [4]:
class Camada:
    def __init__(self, entradas, saidas):
        self.neuronios=[Neuronio(entradas) for _ in range(saidas)]

    def __call__(self, x):
        return [n(x) for n in self.neuronios]


<font color=green> Finalmente podemos criar a classe `MLP` que representa a nossa rede neural de percepção multicamadas. Ela tem duas camadas: a primeira recebe 2 entradas e gera 3 saídas, e a segunda recebe essas 3 saídas e gera 1 resultado final. Quando a rede é chamada com dados de entrada, ela passa essas informações pela primeira camada, depois pela segunda, e retorna a saída final como resultado.

In [5]:
class MLP:
    def __init__(self):
        self.camada1=Camada(2, 3)
        self.camada2=Camada(3, 1)

    def __call__(self, x):
        out1=self.camada1(x)
        out2=self.camada2(out1)
        return out2[0]

<font color=green> Agora que já definimos as classes necessárias para treinar a nossa rede, nós podemos realizar o treinamento da rede e analizar seu desempenho.

In [6]:
rede=MLP()
taxa_aprendizado=0.1

for epoca in range(100):
    erro_total=0
    for xi, yi in zip(X_train, y_train):
        entrada=[Valor(x) for x in xi]
        saida=rede(entrada)
        erro=(saida.data-yi)**2
        erro_total+=erro

        delta=saida.data-yi
        for camada in [rede.camada2, rede.camada1]:
            for neuronio in camada.neuronios:
                for i in range(len(neuronio.pesos)):
                    neuronio.pesos[i].data-=taxa_aprendizado*delta*xi[i%2]  # simplificação
                neuronio.bias.data-=taxa_aprendizado*delta

    if epoca %10==0:
        print(f"Época {epoca}, Erro médio: {erro_total/len(X_train):.4f}")

Época 0, Erro médio: 0.0470
Época 10, Erro médio: 0.0123
Época 20, Erro médio: 0.0098
Época 30, Erro médio: 0.0085
Época 40, Erro médio: 0.0081
Época 50, Erro médio: 0.0080
Época 60, Erro médio: 0.0080
Época 70, Erro médio: 0.0080
Época 80, Erro médio: 0.0080
Época 90, Erro médio: 0.0080


In [12]:
acertos=0
for xi, yi in zip(X_test, y_test):
    entrada=[Valor(x) for x in xi]
    saida=rede(entrada)
    pred=1 if saida.data > 0.5 else 0
    print(f" Y real: {yi}; Y previsto: {pred};")
    if pred==yi:
        acertos+=1

print(f"Acurácia na base de teste: {acertos/len(y_test)*100:.2f}%")

 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 0; Y previsto: 0;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 0;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 0; Y previsto: 0;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 0; Y previsto: 0;
 Y real: 0; Y previsto: 0;
 Y real: 0; Y previsto: 0;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 0; Y previsto: 0;
 Y real: 1; Y previsto: 1;
 Y real: 0; Y previsto: 0;
 Y real: 1; Y previsto: 1;
 Y real: 1; Y previsto: 1;
 Y real: 0; Y previsto: 0;
 Y real: 1; Y previsto: 1;
 Y real: 0; Y previsto: 0;
Acurácia na base de teste: 96.43%


<font color=green>Podemos perceber que o nosso modelo foi muito eficiente para prever se o geiser é curto(0) ou longo(1). A partir desse notebook podemos perceber algumas diferenças entre um modelo MLP de regressão e um modelo MLP de classificação, principalmente no tipo de saída e na função usada para gerar essa saída. No MLP de regressão, a rede gera um valor numérico contínuo (como prever o preço de uma casa), geralmente sem aplicar funções de ativação na última camada, ou usando funções como a identidade. Já no MLP de classificação, a saída representa categorias (como identificar se uma imagem é de um gato ou cachorro), e a última camada costuma usar funções como softmax (para múltiplas classes) ou sigmoide (para uma classe) para transformar os valores em probabilidades. Portanto, a diferença está no propósito da rede e em como ela trata a saída final.
    
<font color=green> Referências
    
-    https://thiagogglopes97.medium.com/classifica%C3%A7%C3%A3o-ou-regress%C3%A3o-e03c05a662a
-    Material de aula